# Limpel-Ziv-Welch Encoder
This python note book contains code to encode an ASCII mesage into LZW binary code using the basic ASCII table found [here](https://theasciicode.com.ar/)

In [1]:
import pandas as pd
import copy
import no_re as re

## Create the dictionary

In [2]:
dictionary = pd.DataFrame(columns=['Hex', 'Char'])
df = pd.read_csv('ascii-table.csv')
df = df.drop(columns=['Number'])
print(df)

    symbol                                        Description
0      NaN                                   (Null character)
1      SOH                                  (Start of Header)
2      STX                                    (Start of Text)
3      ETX                                      (End of Text)
4      EOT                              (End of Transmission)
..     ...                                                ...
123      {                         (curly brackets or braces)
124      |  (vertical-bar, vbar, vertical line or vertical...
125      }                         (curly brackets or braces)
126      ~                               (Tilde ; swung dash)
127    DEL                                           (Delete)

[128 rows x 2 columns]


Create binary numbers

In [3]:
binary_numbers = []
for i in range(128):
    entry = bin(i)[2:]
    binary_numbers.append(entry)

print(binary_numbers)

['0', '1', '10', '11', '100', '101', '110', '111', '1000', '1001', '1010', '1011', '1100', '1101', '1110', '1111', '10000', '10001', '10010', '10011', '10100', '10101', '10110', '10111', '11000', '11001', '11010', '11011', '11100', '11101', '11110', '11111', '100000', '100001', '100010', '100011', '100100', '100101', '100110', '100111', '101000', '101001', '101010', '101011', '101100', '101101', '101110', '101111', '110000', '110001', '110010', '110011', '110100', '110101', '110110', '110111', '111000', '111001', '111010', '111011', '111100', '111101', '111110', '111111', '1000000', '1000001', '1000010', '1000011', '1000100', '1000101', '1000110', '1000111', '1001000', '1001001', '1001010', '1001011', '1001100', '1001101', '1001110', '1001111', '1010000', '1010001', '1010010', '1010011', '1010100', '1010101', '1010110', '1010111', '1011000', '1011001', '1011010', '1011011', '1011100', '1011101', '1011110', '1011111', '1100000', '1100001', '1100010', '1100011', '1100100', '1100101', '11

Front fill zeros and add them to the dictionary

In [4]:
def front_fill_zeros(binary_number, desired_length):
    difference = desired_length - len(binary_number)
    if difference == 0: return binary_number

    zeros = difference * '0'
    return zeros + binary_number

In [5]:
max_length = max(len(number) for number in binary_numbers)
binary_numbers_new = [front_fill_zeros(i, max_length) for i in binary_numbers]
df['Binary'] = binary_numbers_new
df.at[0, 'symbol'] = 'NULL'
# df.at[10, 'symbol'] = '\n'
print(df)

    symbol                                        Description   Binary
0     NULL                                   (Null character)  0000000
1      SOH                                  (Start of Header)  0000001
2      STX                                    (Start of Text)  0000010
3      ETX                                      (End of Text)  0000011
4      EOT                              (End of Transmission)  0000100
..     ...                                                ...      ...
123      {                         (curly brackets or braces)  1111011
124      |  (vertical-bar, vbar, vertical line or vertical...  1111100
125      }                         (curly brackets or braces)  1111101
126      ~                               (Tilde ; swung dash)  1111110
127    DEL                                           (Delete)  1111111

[128 rows x 3 columns]


In [6]:
test = df.at[10,'symbol']
# test = str(test)

print(test)

LF


## Encoding Functions

In [7]:
ORIGINAL_DICTIONARY_LENGTH = 128

def dictionary_hit(symbol: str, dictionary: pd.DataFrame) -> bool:
    """Checks if 'entry' is in the 'symbol' column for the dictionary. 
    
    **WARNING** flags yes if the the string entry is only a part of a different symbol

    eg: DE will be flagged as a hit as it is part of DEL

    Parameters
    ----------
    symbol : str
        A symbol that may or maynot be in the dictionary
    dictionary : pd.DataFrame
        The dictionary of ASCII symbols

    Returns
    -------
    bool
        True if the symbol is within the dictionary
    """
    
    print(f"This is the symbol -{symbol}-")
    symbol=r"{}".format(symbol)
    return True in dictionary['symbol'].str.contains(symbol, case=False, regex=True).values


def add_to_dictionary(entry: str, dictionary: pd.DataFrame) -> pd.DataFrame:
    """Adds a new symbol (entry) to the ASCII dictionary

    Parameters
    ----------
    entry : str
        The new symbol being added to the dictionary
    dictionary : pd.DataFrame
        The old dictionary

    Returns
    -------
    pd.DataFrame
        The updated dictionary with the new entry at the bottom
    """
    binary_num = bin(dictionary.shape[0])[2:] # [2:] to remove the '0b' from the start of the binary number
    previous_binary_num = dictionary['Binary'].iloc[-1]

    if len(binary_num) != len(previous_binary_num):
        print(f"Transitioning from {len(previous_binary_num)} bits to {len(binary_num)} bits")
        dictionary = back_fill_zeros_dictionary(dictionary)

    new_row = pd.Series([entry, "-", binary_num], index=dictionary.columns).T
    new_row_df = pd.DataFrame(new_row).T
    dictionary = pd.concat([dictionary, new_row_df], ignore_index=True)
    print(dictionary)
    return dictionary


def back_fill_zeros_dictionary(dictionary: pd.DataFrame) -> pd.DataFrame:
    """Adds zeros to the the front of the previous binary numbers in a dataframe, used to transition from n -> n+1 bits per symbol

    Parameters
    ----------
    dictionary : pd.DataFrame
        The dictionary of ASCII symbols

    Returns
    -------
    pd.DataFrame
        Updated dictionary of ASCII symbols
    """
    print("Back filling")
    binaries = dictionary['Binary'].values
    new_length = len(binaries[0]) + 1
    new_binaries = [front_fill_zeros(i, new_length) for i in binaries]
    dictionary['Binary'] = new_binaries
    return dictionary

def get_next_char(message: str, dictionary: pd.DataFrame) -> str:
    """Returns the ASCII symbol at the start of the message

    Parameters
    ----------
    message : str
        Remaining ASCII message being encoded
    dictionary : pd.DataFrame
        The dictionary of ASCII symbols

    Returns
    -------
    str
        The ASCII symbol at the start of the message
    """
    original_dictionary = dictionary[:ORIGINAL_DICTIONARY_LENGTH]
    print(f"In get next char")
    return get_largest_string(message, original_dictionary)

def get_largest_string(message: str, dictionary: pd.DataFrame) -> str:
    """Scans the message from the first character in the message for largest sequence present in the dictionary

    Parameters
    ----------
    message : str
        Remaining ASCII message being encoded
    dictionary : pd.DataFrame
        The dictionary of ASCII symbols

    Returns
    -------
    str
        The largest sequence in the dictionary
    """

    i = 1

    while dictionary_hit(message[:i], dictionary) and i <= len(message):
        print(f"mesage i -{message[:i]}-")
        i = i + 1
    i = i - 1

    # Check that the symbol is in the dictionary, fixes message DE being flagged as DEL
    while not message[:i] in dictionary['symbol'].values:
        i = i - 1
        # print(message[:i])
        # print(i)

    return message[:i]


def get_binary(sequence: str, dictionary: pd.DataFrame) -> str:
    """Returns the binary string for a symbol in the dictionary

    Parameters
    ----------
    sequence : str
        The symbol within the dictionary
    dictionary : pd.DataFrame
        The ASCII dictionary

    Returns
    -------
    str
        The binary string with in the dictionary
    """
    row = dictionary.loc[dictionary['symbol'] == sequence]
    return row['Binary'].values[0]

def encode(message: str, dictionary: pd.DataFrame) -> str:
    """The main function to encode a message into binary LZW code

    Parameters
    ----------
    message : str
        The raw ASCII message being encoded
    dictionary : pd.DataFrame
        The dictionary of ASCII symbols
    """
    message_copy = copy.copy(message)

    binary_message = ""

    while message_copy != "":
        # print(f"Remaing message: {message_copy}")
        current_seq = get_largest_string(message_copy, dictionary)
        # print(f"Largest Sequence: {current_seq}")
        output_bits = get_binary(current_seq, dictionary)
        # print(f"Output bits: {output_bits}")
        binary_message = binary_message + output_bits
        message_copy = message_copy.replace(current_seq, "", 1)
        
        if message_copy == "":
            continue

        next_char = get_next_char(message_copy, dictionary)
        # print(f"Next character in the message: {next_char}")

        dictionary_addition = current_seq + next_char
        # print(f"Adding {dictionary_addition} to dictionary")
        dictionary = add_to_dictionary(dictionary_addition, dictionary)
        # print(dictionary)
    
    return binary_message

## Usage

In [8]:
dictionary_hit(" ", df)
ascii_file_path = 'ascii_message.txt'
with open(ascii_file_path) as f:
    message = f.read()

message = message.encode('ascii')
print(type(message))

This is the symbol - -
<class 'bytes'>


In [9]:
ascii_file_path = 'ascii_message.txt'
with open(ascii_file_path) as f:
    message = f.read()

print(message)
message = message.encode('ascii')
message = str(message)[2:-1]
print(message)
message = message.replace('\\n', 'LF')
message = message.replace('[', 'SQO')
message = message.replace(']', 'SQC')
message = message.replace('(', 'RBO')
message = message.replace(')', 'RBC')
print(message)

# message = 'asd  asd s'
binary_message = encode(message, df)
print(f"\nFinal message: {binary_message}")
lzw_file_path = 'lzw_message.txt'
print(type(binary_message))

binary_message = binary_message.encode('utf-8')

with open(lzw_file_path, 'wb') as f:
    f.write(binary_message)

a
Communication system design requires the design of hardware and software systems,
with the objective of transmitting large volumes of information over long
distances, with as few errors as is possible - while all the time consuming as little
power and bandwidth as is possible. This means that impairments such as channel
distortion, interference signals intercepted by the receiver, as well as hardware
impairments at both the transmitter and receiver, requires mitigation [1].
One hardware impairment that is always present at the receiver is thermal
noise, inherent in all materials including conductors, unless cooled to very low
temperatures. When we consider data transmission, we mean the transfer of data
(a discrete bitstream or perhaps a digitized and compressed analog signal) over
a point-to-point, point-to-multipoint or multipoint-to-multipoint communication
channel [2]. Often multipoint channels are based on multiple antennas at either or
both the transmitter and receiver.
a\nComm


This is the symbol -n -
mesage i -n -
This is the symbol -n o-
In get next char
This is the symbol -o-
mesage i -o-
This is the symbol -of-
    symbol            Description    Binary
0     NULL       (Null character)  00000000
1      SOH      (Start of Header)  00000001
2      STX        (Start of Text)  00000010
3      ETX          (End of Text)  00000011
4      EOT  (End of Transmission)  00000100
..     ...                    ...       ...
167     e                       -  10100111
168     de                      -  10101000
169    esi                      -  10101001
170    ign                      -  10101010
171    n o                      -  10101011

[172 rows x 3 columns]
This is the symbol -o-
mesage i -o-
This is the symbol -of-
In get next char
This is the symbol -f-
mesage i -f-
This is the symbol -f -
    symbol            Description    Binary
0     NULL       (Null character)  00000000
1      SOH      (Start of Header)  00000001
2      STX        (Start of Text)  000